- Импортировать данные
  - Данные представляют из себя таблицу (будем обозначать ее **X**),
  строки этой таблицы будем называть **наблюдениями**;
  столбцы этой таблицы будем называть **признаками**
  
- Обработать данные (**отдельно** числовые признаки, отдельно категориальные):

    - подготовить и закодировать категориальные признаки (1 hot encoding), то есть все те признаки которые **не являются** числами. Их нужно представить ввиде чисел. Например признак **Расположение** в твоей таблице может принимать 3 значения: *Переферийная зона*, *Срединная зона*, *Центральная зона*. Его нужно закодировать следующим образом: вместо одного признака **нужно создать 3**, каждый из этих признаков может принимать значения 1 или 0. Например новый признак **Переферийная зона** будет принимать значение **1** для районов (наблюдений): *Ленино, Красный строитель, Осташковское шоссе, Южное очаково, Теплый стан, Перово, Бескудниково, Свиблово, Капотня, Выхино, Чертаново, Серевное Очаково, Кунцево* и **0** для всех остальных. Процедуру необходимо повторить для **всех** категориальных признаков.
    
    - Если у тебя есть числовые признаки одной природы, то они обязательно должны быть переведы в единую систему исчисления. Например если есть несколько числовых признаков обозначающих площадь, они все должны быть в **квадратных километрах** или в **гектарах** или в **квадратных миллиметрах** (не важно в чем, главное чтобы в одном и том же). 
    
    - стандартизовать числовые признаки. Для этого для начала необходимо выбрать из нашей таблицы **X** только числовые признаки, они составят новую таблицу **X_num** в которой столькоже строк (наблюдений) сколько и в **X**, но меньше столбцов. Теперь необходимо посчитать среднее значение каждого признака из **X_num** (mean(X_num)) и стандартное отклонение каждого признака (std(X_num)), а затем вычесть из X_num среднее значение, и поделить на стандартное отклонение:$$\frac{(X_{num} - mean(X_{num}))}{std(X_{num})}$$Допустим у нас есть только 3 числовых признака: Размер территории, Площадь зданий, Структура земли и 30 наблюдений, значит размеры нашей таблицы X_num будет **30 на 3**. Для каждого признака мы считаем **свое** среднее значение и **свое** стандартное отклонение.
    
    
    - после того как мы по отдельности обработали числовые и категориальные признаки мы снова объединяем их в одну таблицу (ее размеры заметно вырастут за счет большого числа закодированных категориальных признаков).
    
    
- Кластеризация
    - в обработанном виде наша таблица готова к кластеризации и ничего дополнительного делать с данными не нужно. Необходимо обпределить число кластеров (k) которые алгоритм будет искать в твоих данных, обычно пробуют разные значения. Я думаю что в твоем случае будет разумно выбрать k от 2 до 10 и посмотреть на результаты в каждом случае. Таким образом входными параметрами алгоритма kmeans служат: данные (X) и число кластеров для поиска (k). А выходным результатом: **вектор длины по числу наблюдений**, этот вектор будет содержать столько уникальных значений, какое k ты укажешь. Например у тебя 10 наблюдений и ты ищешь k=3 кластеров, результирующий вектор **может** принять следующий вид: 0000111122 или 2222101010 или 2102100001 надеюсь идея ясна. Одинаковые значения в этом векторе, значат что соответствующие наблюдения лежат в одном кластере. Например в случае 0000111122 первые 4 наблюдения лежат в первом кластере (метка 0), следующие 4 наблюдения лежат во втором (метка 1), последние 2 наблюдения лежат в третьем (метка 2).


In [1]:
import numpy as np # это для предобработки данных
import pandas as pd  # <--- пакет нужный чтобы считывать xls, xlsx, csv и прочее
from sklearn.cluster import KMeans # <--- это чтобы делать кластеризацию

### Ниже я приведу пример 

ах да, вот еще что, нет никакой проблемы с тем чтобы считывать данные из многих таблиц,
сливать их в одну, подчищать помарки, исправлять ошибки, и делать все это из под Python а не
в визуальном редакторе, но для этого нужен технический навык, и использование некоторого числа 
чисто технических функций, которые тяжко описывать вот так в письменном ввиде, скорее под каждый конкретный случай.
Поэтому когда закончишь заполнять свои таблицы, создай где нибудь отдельную в которую сольешь все все все.

In [2]:
# ВАЖНО! Перед тем как импортировать данные, замени все запятые на точки
# Обычно это на вкладке Edit в самом Excel Делается Find & Replace 
# это необходимо чтобы нормально конвертировать числа в числа.

X = pd.read_excel('Vikas_example_data.xls', sheet='Sheet1')

In [3]:
# Поглядим что у нас импортировалось, для этого напишем:

X.head()

# можно написать X.head(10) тогда напечатаются первые 10 строк
# можно написать X тогда он напечатает столько сколько влезет, 
# остальное скроет

,Размер территории га,Тип территории,Окружение,Расположение,Проницаемость км/га,Доступность 10 км2,Доступность 15 км2,Близость к дорогам,Наличие барьеров,наличие соц активности %,...,Площадь зданий га,Коэф застройки,Фактическая стоимость,Качество инф-ры,Загрязнение почвы,Соотвествие ГП,Структура земл шт/га,Административные барьеры,Визуальная привлек,Репутация тер
Магистральные улицы,247.930276,Смешанная,Смешанная 1,Срединная зона,0.141451,0.360470,1.307027,ТТК. Звенигородское шоссе,жд пути,18.48,...,58.5555,0.236177,NaN,NaN,NaN,да,0.685677,0,NaN,NaN
Павелецкая,207.611685,Офисная,Смешанная 1,Центральная зона,0.152755,0.243075,2.100785,ТТК. Садовое,река. жд пути,54.14,...,57.5795,0.277342,NaN,NaN,NaN,да,2.100785,0,NaN,NaN
Варшавское шоссе,161.503059,Офисная,Смешанная 2,Срединная зона,0.145670,0.496522,0.361765,ТТК. Ваошавское шоссе,река. жд пути,49.20,...,34.5952,0.214208,NaN,NaN,NaN,да,944.000000,0,NaN,NaN
Курская,158.212565,Офисная,Смешанная 1,Центральная зона,0.152151,1.247714,3.180483,Садовое,река. жд пути,34.54,...,46.7353,0.295396,NaN,NaN,NaN,да,1.137710,0,NaN,NaN
Силикатные улицы,156.275321,Смешанная,Смешанная 2,Срединная зона,0.126359,0.140612,0.474502,ТТК.Звенигородское шоссе,река. жд пути,5.46,...,33.5071,0.214411,NaN,NaN,NaN,да,0.428731,0,NaN,NaN


In [4]:
X.columns
# напечатаем все названия всех колонок,
# чтобы выбрать из них категорильные
# и числовые

Index(['Размер территории га', 'Тип территории', 'Окружение', 'Расположение',
       'Проницаемость км/га', 'Доступность 10 км2', 'Доступность 15 км2',
       'Близость к дорогам', 'Наличие барьеров', 'наличие соц активности %',
       'ОКН', 'Площадь зданий га', 'Коэф застройки', 'Фактическая стоимость',
       'Качество инф-ры', 'Загрязнение почвы', 'Соотвествие ГП',
       'Структура земл шт/га', 'Административные барьеры',
       'Визуальная привлек', 'Репутация тер'],
      dtype='object')

In [5]:
# Числовые признаки 8 штук
numerical_features = ['Размер территории га', 'Проницаемость км/га',
                'Доступность 10 км2', 'Доступность 15 км2',
               'наличие соц активности %', 'Площадь зданий га',
                'Коэф застройки', 'Структура земл шт/га']  

# Категориальные признаки 5 штук
categorical_features = ['Тип территории', 'Окружение',
                        'Расположение','Наличие барьеров',
                       'ОКН']

# Оставшиеся признаки в таблице были или слишком пустыми, или сложными для обработки
# в качестве примера (такой только 1 и это Близость к дорогам)

# создадим отдельные переменные под числовые и категориальные признаки
X_num = X[numerical_features].astype(float)
X_cat = X[categorical_features]

## Сперва обработаем числовые признаки

напомню что для них нужно посчитать среднее значение и стандартное отклонение 
а затем вычесть среднее и поделить на стандартное отклонение.

В данный момент у тебя не полностью заполнены столбцы  заполнены 
'Доступность 10 км2', 'Доступность 15 км2', 'наличие соц активности %'
поэтому я пока что, только чтобы не выбрасывать их, заполню недостающие значения
нулями.

In [6]:
# напечатаем таблицу с числовыми признаками
X_num

,Размер территории га,Проницаемость км/га,Доступность 10 км2,Доступность 15 км2,наличие соц активности %,Площадь зданий га,Коэф застройки,Структура земл шт/га
Магистральные улицы,247.930276,0.141451,0.360470,1.307027,18.48,58.555500,0.236177,0.685677
Павелецкая,207.611685,0.152755,0.243075,2.100785,54.14,57.579500,0.277342,2.100785
Варшавское шоссе,161.503059,0.145670,0.496522,0.361765,49.20,34.595200,0.214208,944.000000
Курская,158.212565,0.152151,1.247714,3.180483,34.54,46.735300,0.295396,1.137710
Силикатные улицы,156.275321,0.126359,0.140612,0.474502,5.46,33.507100,0.214411,0.428731
Ленино,127.743366,0.097841,0.000000,0.000000,4.97,17.914700,0.140240,0.430551
Боткинский проезд,127.160047,0.099811,0.062654,0.771639,2.09,43.778400,0.344278,0.400000
Алексеевские улицы,123.500593,0.124731,0.149726,0.798193,25.78,28.739200,0.232705,0.858295
Западный порт,119.377294,0.088115,0.825697,4.148083,47.82,36.250700,0.303665,0.510985
Бережковская Набережная,111.605872,0.071500,0.306098,0.000020,18.34,25.956800,0.232576,0.230000


In [7]:
# Заполним недостающие (NaN) значения нулями,
# у тебя пустых полей по идее быть не должно

X_num.fillna(0, inplace=True)
X_num

,Размер территории га,Проницаемость км/га,Доступность 10 км2,Доступность 15 км2,наличие соц активности %,Площадь зданий га,Коэф застройки,Структура земл шт/га
Магистральные улицы,247.930276,0.141451,0.360470,1.307027,18.48,58.555500,0.236177,0.685677
Павелецкая,207.611685,0.152755,0.243075,2.100785,54.14,57.579500,0.277342,2.100785
Варшавское шоссе,161.503059,0.145670,0.496522,0.361765,49.20,34.595200,0.214208,944.000000
Курская,158.212565,0.152151,1.247714,3.180483,34.54,46.735300,0.295396,1.137710
Силикатные улицы,156.275321,0.126359,0.140612,0.474502,5.46,33.507100,0.214411,0.428731
Ленино,127.743366,0.097841,0.000000,0.000000,4.97,17.914700,0.140240,0.430551
Боткинский проезд,127.160047,0.099811,0.062654,0.771639,2.09,43.778400,0.344278,0.400000
Алексеевские улицы,123.500593,0.124731,0.149726,0.798193,25.78,28.739200,0.232705,0.858295
Западный порт,119.377294,0.088115,0.825697,4.148083,47.82,36.250700,0.303665,0.510985
Бережковская Набережная,111.605872,0.071500,0.306098,0.000020,18.34,25.956800,0.232576,0.230000


In [8]:
# Посчитаем среднее значение и стандартное отклонение
X_num.mean()

Размер территории га        199.793727
Проницаемость км/га           0.111509
Доступность 10 км2            0.102965
Доступность 15 км2            0.316738
наличие соц активности %      8.473830
Площадь зданий га            45.350415
Коэф застройки                0.252416
Структура земл шт/га         20.699181
dtype: float64

In [9]:
X_num.std()

Размер территории га        168.598574
Проницаемость км/га           0.043388
Доступность 10 км2            0.240024
Доступность 15 км2            0.820798
наличие соц активности %     14.831454
Площадь зданий га            33.035509
Коэф застройки                0.106195
Структура земл шт/га        137.605446
dtype: float64

In [10]:
# Как видишь они сразу считаются для каждого столбца по отдельности,
# что нам и нужно!

X_num = (X_num - X_num.mean()) / X_num.std()

In [11]:
# Отлично, с числовыми признаками мы закончили!
X_num.head()

,Размер территории га,Проницаемость км/га,Доступность 10 км2,Доступность 15 км2,наличие соц активности %,Площадь зданий га,Коэф застройки,Структура земл шт/га
Магистральные улицы,0.285510,0.690095,1.072829,1.206496,0.674659,0.399724,-0.152910,-0.145441
Павелецкая,0.046370,0.950622,0.583733,2.173552,3.079008,0.370180,0.234725,-0.135157
Варшавское шоссе,-0.227111,0.787326,1.639654,0.054858,2.745932,-0.325565,-0.359789,6.709769
Курская,-0.246628,0.936706,4.769302,3.488976,1.757493,0.041921,0.404727,-0.142156
Силикатные улицы,-0.258118,0.342246,0.156847,0.192208,-0.203205,-0.358503,-0.357877,-0.147308


## Время поработать с категориальными

В данном случае перекодировку всех числовых признаков сделать просто,
кроме признака Наличие барьеров, потому что он содержит например значения
**жд пути**, и значение **река, жд пути**. Его по хорошему нужно
обрабатывать несколько иначе, но пока и так сойдет

In [12]:
X_cat
# напечатаем чтобы посмотреть только категориальные признаки

,Тип территории,Окружение,Расположение,Наличие барьеров,ОКН
Магистральные улицы,Смешанная,Смешанная 1,Срединная зона,жд пути,нет
Павелецкая,Офисная,Смешанная 1,Центральная зона,река. жд пути,есть
Варшавское шоссе,Офисная,Смешанная 2,Срединная зона,река. жд пути,есть
Курская,Офисная,Смешанная 1,Центральная зона,река. жд пути,есть
Силикатные улицы,Смешанная,Смешанная 2,Срединная зона,река. жд пути,нет
Ленино,Промышленная,Смешанная 3,Периферийная зона,жд пути,нет
Боткинский проезд,Промышленная,Смешанная 3,Срединная зона,нет,есть
Алексеевские улицы,Смешанная,Смешанная 3,Срединная зона,жд пути,есть
Западный порт,Смешанная,Жилая застройка,Центральная зона,река. жд пути,есть
Бережковская Набережная,Инфраструктурная,Жилая застройка,Центральная зона,река. жд пути,есть


In [13]:
X_cat_1hot = pd.get_dummies(X_cat)

In [14]:
# вот так выглядят теперь наши новые категорильные признаки
X_cat_1hot

,Тип территории_Деловая,Тип территории_Жилая,Тип территории_Инфраструктурная,Тип территории_Коммунальная,Тип территории_Офисная,Тип территории_Промышленная,Тип территории_Смешанная,Тип территории_Транспортная,Окружение_Жилая застройка,Окружение_Смешанная 1,...,Расположение_Центральная зона,Наличие барьеров_жд пути,Наличие барьеров_жд пути. промка,Наличие барьеров_нет,Наличие барьеров_промка,Наличие барьеров_пруд. жд пути,Наличие барьеров_река,Наличие барьеров_река. жд пути,ОКН_есть,ОКН_нет
Магистральные улицы,0,0,0,0,0,0,1,0,0,1,...,0,1,0,0,0,0,0,0,0,1
Павелецкая,0,0,0,0,1,0,0,0,0,1,...,1,0,0,0,0,0,0,1,1,0
Варшавское шоссе,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,1,1,0
Курская,0,0,0,0,1,0,0,0,0,1,...,1,0,0,0,0,0,0,1,1,0
Силикатные улицы,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,1,0,1
Ленино,0,0,0,0,0,1,0,0,0,0,...,0,1,0,0,0,0,0,0,0,1
Боткинский проезд,0,0,0,0,0,1,0,0,0,0,...,0,0,0,1,0,0,0,0,1,0
Алексеевские улицы,0,0,0,0,0,0,1,0,0,0,...,0,1,0,0,0,0,0,0,1,0
Западный порт,0,0,0,0,0,0,1,0,1,0,...,1,0,0,0,0,0,0,1,1,0
Бережковская Набережная,0,0,1,0,0,0,0,0,1,0,...,1,0,0,0,0,0,0,1,1,0


In [15]:
# поглядим на названия колонок,
# как видишь они собраны из названия первичного категорильного признака
# и конкретного его значения
X_cat_1hot.columns

Index(['Тип территории_Деловая', 'Тип территории_Жилая',
       'Тип территории_Инфраструктурная', 'Тип территории_Коммунальная',
       'Тип территории_Офисная', 'Тип территории_Промышленная',
       'Тип территории_Смешанная', 'Тип территории_Транспортная',
       'Окружение_Жилая застройка', 'Окружение_Смешанная 1',
       'Окружение_Смешанная 2', 'Окружение_Смешанная 3',
       'Расположение_Периферийная зона', 'Расположение_Срединная зона',
       'Расположение_Центральная зона', 'Наличие барьеров_жд пути',
       'Наличие барьеров_жд пути. промка', 'Наличие барьеров_нет',
       'Наличие барьеров_промка', 'Наличие барьеров_пруд. жд пути',
       'Наличие барьеров_река', 'Наличие барьеров_река. жд пути', 'ОКН_есть',
       'ОКН_нет'],
      dtype='object')

## Время собрать все воедино!

Напомню что числовые признаки у нас лежавт в переменной X_num, а закодированные категориальные в переменной X_cat_1hot

In [16]:
X_ready = X_num.join(X_cat_1hot)

In [17]:
X_ready.head()

,Размер территории га,Проницаемость км/га,Доступность 10 км2,Доступность 15 км2,наличие соц активности %,Площадь зданий га,Коэф застройки,Структура земл шт/га,Тип территории_Деловая,Тип территории_Жилая,...,Расположение_Центральная зона,Наличие барьеров_жд пути,Наличие барьеров_жд пути. промка,Наличие барьеров_нет,Наличие барьеров_промка,Наличие барьеров_пруд. жд пути,Наличие барьеров_река,Наличие барьеров_река. жд пути,ОКН_есть,ОКН_нет
Алексеевские улицы,-0.452514,0.304731,0.194818,0.586570,1.166856,-0.502829,-0.185607,-0.144187,0,0,...,0,1,0,0,0,0,0,0,1,0
Бережковская Набережная,-0.523064,-0.922139,0.846302,-0.385866,0.665219,-0.587054,-0.186826,-0.148753,0,0,...,1,0,0,0,0,0,0,1,1,0
Бескудниково,-0.602982,-1.866501,-0.428977,-0.385890,-0.571342,-0.808991,-0.589669,-0.146499,0,0,...,0,1,0,0,0,0,0,0,0,1
Боткинский проезд,-0.430808,-0.269617,-0.167945,0.554218,-0.430425,-0.047586,0.865033,-0.147517,0,0,...,0,0,0,1,0,0,0,0,1,0
Братцево,0.172026,1.293047,-0.428977,-0.385890,-0.571342,0.100936,-0.373174,-0.146359,0,0,...,0,1,0,0,0,0,0,0,1,0


## Теперь можно и кластеризацией заняться!

In [18]:
#Результаты кластеризации с разным числом кластеров сохранены в разные переменные

kmeans3 = KMeans(n_clusters = 3)
res3 = kmeans3.fit_predict(X_ready)[np.newaxis,:]

kmeans4 = KMeans(n_clusters = 4)
res4 = kmeans4.fit_predict(X_ready)[np.newaxis,:]

kmeans5 = KMeans(n_clusters = 5)
res5 = kmeans5.fit_predict(X_ready)[np.newaxis,:]

kmeans6 = KMeans(n_clusters = 6)
res6 = kmeans6.fit_predict(X_ready)[np.newaxis,:]

kmeans7 = KMeans(n_clusters = 7)
res7 = kmeans7.fit_predict(X_ready)[np.newaxis,:]

kmeans8 = KMeans(n_clusters = 8)
res8 = kmeans8.fit_predict(X_ready)[np.newaxis,:]

# Соберем все переменные в одну
clustering_result = np.concatenate((res3, res4, res5, res6, res7, res8), axis=0)

In [19]:
# чтобы понять что к чему создадим новую таблицу
# в которой строки это Названия Района, а столбцы - результаты кластеризации

res_column_names = ['k=3','k=4','k=5',
                    'k=6','k=7','k=8']
res_df = pd.DataFrame(data =clustering_result.T,
                      index=X_ready.index, columns=res_column_names)

In [20]:
res_df

,k=3,k=4,k=5,k=6,k=7,k=8
Алексеевские улицы,0,3,1,0,5,3
Бережковская Набережная,0,3,1,0,5,3
Бескудниково,0,3,1,2,4,7
Боткинский проезд,0,3,1,0,1,7
Братцево,0,3,1,0,1,2
Варшавское шоссе,1,1,3,3,6,6
Верхние котлы,0,3,1,2,1,0
Владыкино,0,3,1,2,1,7
Волгоградский проспект,2,0,0,5,0,4
Воронцово,2,0,0,5,1,2


In [21]:
# Теперь можно например выбрать какое то k (например 5)
# и посмотреть все районы с меткой 0 

res_df[res_df['k=5']==0]['k=5']

Волгоградский проспект    0
Воронцово                 0
Грайвороново              0
ЗИЛ                       0
Капотня                   0
Огородный проезд          0
Северянин                 0
Серевное Очаково          0
Серп и молот              0
Чертаново                 0
Южный порт                0
Name: k=5, dtype: int32

In [22]:
# с меткой 1 
res_df[res_df['k=5']==1]['k=5']

Алексеевские улицы         1
Бережковская Набережная    1
Бескудниково               1
Боткинский проезд          1
Братцево                   1
Верхние котлы              1
Владыкино                  1
Выхино                     1
Грузинский вал             1
Донские улицы              1
Звенигородское шоссе       1
Калибр                     1
Каширское шоссе            1
Кирпичные улицы            1
Красный строитель          1
Кунцево                    1
Ленино                     1
Магистральные улицы        1
Митьковская ветка          1
Нагатино                   1
Октябрьское поле           1
Осташковское шоссе         1
Перово                     1
Площадь развилки           1
Свиблово                   1
Силикатные улицы           1
Соколиная гора             1
Теплый стан                1
Улица Правды               1
Хапиловка                  1
Хапиловка                  1
Южное очаково              1
Name: k=5, dtype: int32

In [23]:
res_df[res_df['k=5']==2]['k=5']

Западный порт    2
Курская          2
Павелецкая       2
Name: k=5, dtype: int32

In [24]:
res_df[res_df['k=5']==3]['k=5']

Варшавское шоссе    3
Name: k=5, dtype: int32

In [25]:
res_df[res_df['k=5']==4]['k=5']
# В строках у тебя почему то две Хапиловки,
# наверное просто опечатка

Хапиловка    4
Хапиловка    4
Name: k=5, dtype: int32